## Downloading NEXRAD data
Scans are an available open data source: https://www.ncdc.noaa.gov/data-access/radar-data/noaa-big-data-project
They are stored in AWS warehousing by year, month, date, and time of scan for each radar location. 

Documentation: https://docs.opendata.aws/noaa-nexrad/readme.html
code adopted from https://www1.ncdc.noaa.gov/pub/data/radar/bdp/scripts/

Website to get sunrise time: https://www.esrl.noaa.gov/gmd/grad/solcalc/sunrise.html

In [1]:
import sys
import urllib.request
from xml.dom import minidom
from sys import stdin
from urllib.request import urlopen
from subprocess import call
import datetime
import pandas as pd
import json

sys.path.append('../')
from envir import config

In [2]:
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

In [16]:
date_range = pd.date_range(start="2017/05/17", end="2017/05/19").astype(str)
#date_range = date_range.to_series().apply(lambda x: x.replace("-", "/"))

In [17]:
# from google maps
# https://www.google.com/maps/place/Washington+Square+Park/@40.730959,-73.9987294,18z/data=!4m12!1m6!3m5!1s0x0:0xfa615edfd1b67e18!2sWashington+Square+Park!8m2!3d40.7308228!4d-73.997332!3m4!1s0x0:0xfa615edfd1b67e18!8m2!3d40.7308228!4d-73.997332
sundoc = urllib.request.urlopen('https://api.sunrise-sunset.org/json?lat=40.730959&lng=-73.9987294&date='+'2017-05-17')
sundoc = json.loads(sundoc.read())

In [28]:
#### script to download sunrise and sunset time each day in Washington Square Park, NYC
import sys
import json
import urllib
from urllib.request import urlopen
import datetime
import pandas as pd
from pandas.io.json import json_normalize

sys.path.append('../')
from envir import config

####

date_range = pd.date_range(start="2017/05/17", end="2017/05/19").astype(str)
# from google maps
# https://www.google.com/maps/place/Washington+Square+Park/@40.730959,-73.9987294
lat = '40.730959'
long = '-73.9987294'

dailySunDoc = pd.DataFrame()

for day in date_range:
    sundoc = urllib.request.urlopen('https://api.sunrise-sunset.org/json?lat='+
                                    lat+'&lng='+long+'&date='+day)
    print('https://api.sunrise-sunset.org/json?lat='+
                                    lat+'&lng='+long+'&date='+day)
    sundoc = json.loads(sundoc.read())
    tempSunDoc = json_normalize(sundoc['results'])
    tempSunDoc['date'] = day
    dailySunDoc = dailySunDoc.append(tempSunDoc, ignore_index=True)
    
dailySunDoc

https://api.sunrise-sunset.org/json?lat=40.730959&lng=-73.9987294&date=2017-05-17
https://api.sunrise-sunset.org/json?lat=40.730959&lng=-73.9987294&date=2017-05-18
https://api.sunrise-sunset.org/json?lat=40.730959&lng=-73.9987294&date=2017-05-19


,sunrise,sunset,solar_noon,day_length,civil_twilight_begin,civil_twilight_end,nautical_twilight_begin,nautical_twilight_end,astronomical_twilight_begin,astronomical_twilight_end,date
0,9:36:02 AM,12:08:48 AM,4:52:25 PM,14:32:46,9:04:42 AM,12:40:08 AM,8:25:36 AM,1:19:14 AM,7:41:43 AM,2:03:07 AM,2017-05-17
1,9:35:11 AM,12:09:45 AM,4:52:28 PM,14:34:34,9:03:45 AM,12:41:10 AM,8:24:29 AM,1:20:27 AM,7:40:19 AM,2:04:36 AM,2017-05-18
2,9:34:21 AM,12:10:41 AM,4:52:31 PM,14:36:20,9:02:50 AM,12:42:12 AM,8:23:24 AM,1:21:38 AM,7:38:57 AM,2:06:05 AM,2017-05-19


In [17]:
for i in date_range:
    date = i
    site = "KOKX"
    bucketURL = "http://noaa-nexrad-level2.s3.amazonaws.com"
    dirListURL = bucketURL+ "/?prefix=" + date + "/" + site
    print("listing files from %s" % dirListURL)
    
    #xmldoc = minidom.parse(stdin)
    xmldoc = minidom.parse(urlopen(dirListURL))
    itemlist = xmldoc.getElementsByTagName('Key')
    print(len(itemlist) , "keys found...")
    
    # For this test, WCT is downloaded and unzipped directly in the working directory
    # The output files are going in 'output'
    # http://www.ncdc.noaa.gov/wct/install.php
    for x in itemlist:
        file = getText(x.childNodes)
        print("Found %s " % file)
        print(file[-10:-8])

listing files from http://noaa-nexrad-level2.s3.amazonaws.com/?prefix=2017/05/17/KOKX
183 keys found...
Found 2017/05/17/KOKX/KOKX20170517_000755_V06 
00
Found 2017/05/17/KOKX/KOKX20170517_001736_V06 
00
Found 2017/05/17/KOKX/KOKX20170517_002717_V06 
00
Found 2017/05/17/KOKX/KOKX20170517_003910_V06 
00
Found 2017/05/17/KOKX/KOKX20170517_004851_V06 
00
Found 2017/05/17/KOKX/KOKX20170517_005834_V06 
00
Found 2017/05/17/KOKX/KOKX20170517_010815_V06 
01
Found 2017/05/17/KOKX/KOKX20170517_011758_V06 
01
Found 2017/05/17/KOKX/KOKX20170517_012741_V06 
01
Found 2017/05/17/KOKX/KOKX20170517_013724_V06 
01
Found 2017/05/17/KOKX/KOKX20170517_014705_V06 
01
Found 2017/05/17/KOKX/KOKX20170517_015647_V06 
01
Found 2017/05/17/KOKX/KOKX20170517_020629_V06 
02
Found 2017/05/17/KOKX/KOKX20170517_021611_V06 
02
Found 2017/05/17/KOKX/KOKX20170517_022553_V06 
02
Found 2017/05/17/KOKX/KOKX20170517_023536_V06 
02
Found 2017/05/17/KOKX/KOKX20170517_024518_V06 
02
Found 2017/05/17/KOKX/KOKX20170517_025501_V06 

In [2]:
print('Beginning file download with urllib2...')

url = 'https://noaa-nexrad-level2.s3.amazonaws.com/2017/04/01/KOKX/KOKX20170401_063244_V06'
urllib.request.urlretrieve(url, config.dataFol+'nexrad_downloads/KOKX20170401_063244_V06.h5')

Beginning file download with urllib2...


('/gws_gpfs/projects/project-urban_birds/workspace/share/data/nexrad_downloads/KOKX20170401_063244_V06.h5',
 <http.client.HTTPMessage at 0x7ff7f5118c90>)